In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score,accuracy_score

X,y = datasets.load_iris(return_X_y=True)
print(type(X))
print(X.shape,y.shape)
scoring = ['precision_macro','recall_macro','accuracy']
classifier = svm.SVC(kernel='linear',C=1,random_state=10)
scores = cross_validate(classifier,X,y,scoring=scoring,cv=4)
print(scores)

<class 'numpy.ndarray'>
(150, 4) (150,)
{'fit_time': array([0.00406837, 0.01268077, 0.00132465, 0.00553656]), 'score_time': array([0.00244713, 0.00239849, 0.00931883, 0.00243211]), 'test_precision_macro': array([1.        , 0.97619048, 0.97435897, 0.97619048]), 'test_recall_macro': array([1.        , 0.97222222, 0.97435897, 0.97222222]), 'test_accuracy': array([1.        , 0.97368421, 0.97297297, 0.97297297])}


In [27]:
from sklearn.model_selection import RepeatedKFold
random_state = 12883823

rkf = RepeatedKFold(n_splits=3, n_repeats=2, random_state=random_state)

#non funge
for result in rkf.scoring:
    print(result)

AttributeError: 'RepeatedKFold' object has no attribute 'scoring'

In [33]:

risultati = []

for i in range(1,4):
    risultati.append(cross_validate(classifier,X,y,scoring=scoring,cv=4))

for i in range(0,3):
    print(1 - risultati[i]['test_accuracy'])

[0.         0.02631579 0.02702703 0.02702703]
[0.         0.02631579 0.02702703 0.02702703]
[0.         0.02631579 0.02702703 0.02702703]


In [51]:
#in effetti lo shuffle si limita a fare una corretta permutazione
#senza perdere corrispondenze tra X e Y
from sklearn.utils import shuffle

ar1 = np.array([[1, 3],[1, 4],[2, 4],[3, 1],[1,6]])
ar2 = np.array([0,1,0,0,1])

print(ar1.shape,ar2.shape)

for i in range(1,4):
    ar1,ar2 = shuffle(ar1,ar2,random_state=1234*i)
    print(ar1,ar2)

(5, 2) (5,)
[[1 6]
 [1 3]
 [1 4]
 [2 4]
 [3 1]] [1 0 1 0 0]
[[3 1]
 [1 3]
 [2 4]
 [1 6]
 [1 4]] [0 0 0 1 1]
[[3 1]
 [1 6]
 [2 4]
 [1 3]
 [1 4]] [0 1 0 0 1]


In [70]:
#calcoliamo il t-student
#2)fare 5 modelli (quelli di Ale ;)
X,y
results_A = []
results_B = []
model_A ={"Accuracy":[]}
model_B ={"Accuracy":[]}
for i in range(1,4):
    Xa,ya = shuffle(X,y,random_state=1234*i)
    Xb,yb = shuffle(X,y,random_state=4321*i)
    
    results_A.append(cross_validate(classifier,Xa,ya,scoring=scoring,cv=4))
    results_B.append(cross_validate(classifier,Xb,yb,scoring=scoring,cv=4))

for i in range(0,3):
    print(results_A[i]['test_accuracy'])
    print(results_B[i]['test_accuracy'])
print("---------------------------------")    
for result in results_A:
    model_A["Accuracy"].append(np.mean(result['test_accuracy']))
for result in results_B:
    model_B["Accuracy"].append(np.mean(result['test_accuracy']))
    
print(model_A["Accuracy"])
print(model_B["Accuracy"])

[0.97368421 1.         0.94594595 1.        ]
[0.97368421 0.97368421 0.97297297 1.        ]
[1.         0.94736842 1.         0.94594595]
[0.97368421 0.97368421 1.         0.94594595]
[0.97368421 0.94736842 0.94594595 0.97297297]
[0.97368421 0.97368421 0.97297297 0.97297297]
---------------------------------
[0.9799075391180655, 0.9733285917496444, 0.9599928876244666]
[0.9800853485064012, 0.9733285917496445, 0.9733285917496444]


In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score,accuracy_score

X,y = datasets.load_iris(return_X_y=True)
print(type(X))
print(X.shape,y.shape)
scoring = ['precision_macro','recall_macro','accuracy']
classifier = svm.SVC(kernel='linear',C=1,random_state=10)


import math
from sklearn.utils import shuffle

n_rounds = 10 #degree of freedom 9 (= 10-1)
err_A = []
err_B = []
overall_err_A = 0.0
overall_err_B = 0.0

results_A = []
results_B = []
model_A ={"Accuracy":[]}
model_B ={"Accuracy":[]}

for rounds in range(1,n_rounds+1):
    Xa,ya = shuffle(X,y,random_state=1234*rounds)
    Xb,yb = shuffle(X,y,random_state=4321*rounds)#da eliminare nella versione finale
    
    results_A.append(cross_validate(classifier,Xa,ya,scoring=scoring,cv=10))
    #results_B.append(cross_validate(classifier,Xa,ya,scoring=scoring,cv=10)) #codice vero
    results_B.append(cross_validate(classifier,Xb,yb,scoring=scoring,cv=10)) #codice for testing

for result in results_A:
    model_A["Accuracy"].append(np.mean(result['test_accuracy']))
for result in results_B:
    model_B["Accuracy"].append(np.mean(result['test_accuracy']))

err_A = 1 - np.array(model_A["Accuracy"])
err_B = 1 - np.array(model_B["Accuracy"])

print(err_A)
print(err_B)

overall_err_A = np.mean(err_A)
overall_err_B = np.mean(err_B)

print(overall_err_A)
print(overall_err_B)


partial_varAB = 0.0
for i in range(0,n_rounds):
    partial_varAB = partial_varAB + pow((err_A[i]-err_B[i]-(overall_err_A-overall_err_B)),2)

varAB=1/n_rounds * partial_varAB

t_test = (overall_err_A-overall_err_B)/math.sqrt(varAB/n_rounds)

print(t_test)

#t-table
t_table = pd.read_csv("t-table.csv")
t_distribution = t_table.loc[9,"0.025"]

print("t-distribution value is:{}".format(t_distribution))
if(t_test < -t_distribution or t_test > t_distribution):
    print("The Null Hypothesis can be rejected. The two classifiers are statistically significant different.")
else:
    print("The Null Hypothesis can not be rejected. The two classifiers are statistically different by the chance.")

<class 'numpy.ndarray'>
(150, 4) (150,)
[0.02       0.02       0.02       0.04       0.02666667 0.02
 0.02       0.01333333 0.02666667 0.02      ]
[0.02666667 0.02666667 0.02       0.02       0.02       0.01333333
 0.02       0.02       0.02       0.02      ]
0.022666666666666602
0.02066666666666659
0.7989354619369626
t-distribution value is:2.262
The Null Hypothesis can not be rejected. The two classifiers are statistically different by the chance.


In [16]:



#FUNZIONE DEFINITIVA. 
#Ricorda di importare le opportune librerie!!

def pairwiseComparisionModel(pipeline_A,pipeline_B,dataset,labels,n_rounds):
    if(n_rounds <= 1):
        print("Errore:number of rounds at least 2")
        return 
    
        
    degFredoom = n_rounds - 1 #degrees of freedom
    err_A = []
    err_B = []
    overall_err_A = 0.0
    overall_err_B = 0.0

    results_A = []
    results_B = []
    model_A ={"Accuracy":[]}
    model_B ={"Accuracy":[]}
    
    for rounds in range(1,n_rounds+1):
        Xa,ya = shuffle(X,y,random_state=123456789*rounds)
        results_A.append(cross_validate(estimator=pipeline_A,X=Xa,y=ya,scoring=scoring,cv=10))
        results_B.append(cross_validate(estimator=pipeline_B,X=Xa,y=ya,scoring=scoring,cv=10))
    
    for result in results_A:
        model_A["Accuracy"].append(np.mean(result['test_accuracy']))
    for result in results_B:
        model_B["Accuracy"].append(np.mean(result['test_accuracy']))

    err_A = 1 - np.array(model_A["Accuracy"])
    err_B = 1 - np.array(model_B["Accuracy"])

    print(err_A)
    print(err_B)

    overall_err_A = np.mean(err_A)
    overall_err_B = np.mean(err_B)

    print(overall_err_A)
    print(overall_err_B)


    partial_varAB = 0.0
    for i in range(0,n_rounds):
        partial_varAB = partial_varAB + pow((err_A[i]-err_B[i]-(overall_err_A-overall_err_B)),2)

    varAB=1/n_rounds * partial_varAB

    t_test = (overall_err_A-overall_err_B)/math.sqrt(varAB/n_rounds)

    print(t_test)

    #t-table
    t_table = pd.read_csv("t-table.csv")
    t_distribution = t_table.loc[degFreedom,"0.025"]

    print("t-distribution value is:{}".format(t_distribution))
    if(t_test < -t_distribution or t_test > t_distribution):
        print("The Null Hypothesis can be rejected. The two classifiers are statistically significant different.")
    else:
        print("The Null Hypothesis can not be rejected. The two classifiers are statistically different by the chance.")